In [0]:
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")

In [0]:
dbutils.widgets.text("p_file_date", "")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/paths"

In [0]:
%run "../includes/common_functions"

In [0]:
from pyspark.sql.types import *

name_schema = StructType([StructField("forename", StringType(), True), StructField("surname", StringType(), True)])

drivers_schema = StructType([StructField("code", StringType(), True),
                            StructField("dob", DateType(), True), 
                            StructField("driverId", IntegerType(), True), 
                            StructField("driverRef", StringType(), True), 
                            StructField("name", name_schema), 
                            StructField("nationality", StringType(), True), 
                            StructField("number", IntegerType(), True), 
                            StructField("url", StringType(), True)])

In [0]:
drivers_df = spark.read.schema(drivers_schema).json(f"{raw_container_folder_path}/{v_file_date}/drivers.json")

In [0]:
from pyspark.sql.functions import *

drivers_final_df = drivers_df.withColumn("name", concat(col("name.forename"), lit(" "), col("name.surname")))\
                                .withColumnRenamed("driverId","driver_id")\
                                .withColumnRenamed("driverRef","driver_ref")\
                                .withColumn("ingestion_date", current_timestamp())\
                                .withColumn("data_source", lit(v_data_source))\
                                .withColumn("file_date", lit(v_file_date))\
                                .drop(col("url"))

In [0]:
#drivers_final_df.write.mode("overwrite").parquet(f"{processed_container_folder_path}/drivers")

drivers_final_df.write.mode("overwrite").format("delta").saveAsTable("f1_processed_db.drivers")

In [0]:
dbutils.notebook.exit("Success")